***GENERATED CODE FOR rideregressionall PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'datetime', 'transformation_label': 'String Indexer'}], 'feature': 'datetime', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '244', 'mean': '', 'stddev': '', 'min': '1/1/2024', 'max': '8/9/2024', 'missing': '0', 'distinct': '244'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'datetime'}, {'feature_label': 'datetime', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('datetime')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run rideregressionallHooks.ipynb
try:
	#sourcePreExecutionHook()

	datastandardizedall = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/data_standardized_2024_all.csv', 'filename': 'data_standardized_2024_all.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run rideregressionallHooks.ipynb
try:
	#transformationPreExecutionHook()

	rideregressionallautofe = TransformationMain.run(datastandardizedall,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "datetime", "transformation_label": "String Indexer"}], "feature": "datetime", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "244", "mean": "", "stddev": "", "min": "1/1/2024", "max": "8/9/2024", "missing": "0", "distinct": "244"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "datetime"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "tempmax", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "0.1", "stddev": "1.08", "min": "-2.442743144", "max": "1.98402119", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "tempmax"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "tempmin", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "0.12", "stddev": "1.05", "min": "-2.384971861", "max": "1.755295464", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "tempmin"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "tempmax_lag1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "0.1", "stddev": "1.08", "min": "-2.441428297", "max": "1.988938028", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "tempmax_lag1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "humidity_lag1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "-0.1", "stddev": "0.99", "min": "-2.362600624", "max": "2.043962387", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "humidity_lag1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "tempmax_rolling3", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "0.1", "stddev": "1.08", "min": "-2.441428297", "max": "1.988938028", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "tempmax_rolling3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "humidity_rolling7", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "-0.11", "stddev": "0.99", "min": "-2.456280425", "max": "2.08821821", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "humidity_rolling7"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "humidity", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "-0.09", "stddev": "1.0", "min": "-2.293973681", "max": "2.035074484", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "humidity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "precip", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "-0.04", "stddev": "0.92", "min": "-0.373893795", "max": "7.00257513", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "precip"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "windspeed", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "0.05", "stddev": "1.0", "min": "-1.926499719", "max": "3.421181608", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "windspeed"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ride", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "1.12", "stddev": "1.01", "min": "-1.574437964", "max": "2.921393714", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ride"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ride_duration", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "-0.12", "stddev": "1.98", "min": "-1.251256547", "max": "24.71342659", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ride_duration"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Season_spring", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "244", "mean": "0.38", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Season_spring"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Season_summer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "244", "mean": "0.38", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Season_summer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Season_winter", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "244", "mean": "0.25", "stddev": "0.43", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Season_winter"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IsWeekend", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "244", "mean": "0.28", "stddev": "0.45", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "IsWeekend"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "244", "mean": "2024.0", "stddev": "0.0", "min": "2024", "max": "2024", "missing": "0"}, "updatedLabel": "Year"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "244", "mean": "4.52", "stddev": "2.3", "min": "1", "max": "8", "missing": "0"}, "updatedLabel": "Month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "WeekNumber", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "244", "mean": "2.7", "stddev": "1.27", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "WeekNumber"}]}))

	#transformationPostExecutionHook(rideregressionallautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run rideregressionallHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(rideregressionallautofe, ["tempmax", "tempmin", "tempmax_lag1", "humidity_lag1", "tempmax_rolling3", "humidity_rolling7", "humidity", "precip", "windspeed", "ride_duration", "Season_spring", "Season_summer", "Season_winter", "IsWeekend", "Year", "Month", "WeekNumber", "datetime_stringindexer"], "ride")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

